In [ ]:
import http.client
from dotenv import load_dotenv
import os
load_dotenv()


conn = http.client.HTTPSConnection("sky-scanner3.p.rapidapi.com")

headers = {
    "Content-Type": "application/json",
    "X-RapidAPI-Key": os.getenv("FLIGHTS_RAPIDAPI_KEY"),  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": os.getenv("FLIGHTS_RAPIDAPI_HOST")
}

conn.request("GET", "/flights/price-calendar-web?fromEntityId=YUL&toEntityId=ABJ&yearMonth=2025-02", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":{"Outbound":"2025-02","Traces":{"0":"202502150222*I*YUL*ABJ*20250228*skyp*PD","1":"202502161741*I*YUL*ABJ*20250226*oojo*AT","2":"202502170017*I*YUL*ABJ*20250227*oojo*AT","3":"202502140301*I*YUL*ABJ*20250219*luft*LH","4":"202502170034*I*YUL*ABJ*20250220*oojo*AT","5":"202502150217*I*YUL*ABJ*20250224*oojo*AT","6":"202502161929*I*YUL*ABJ*20250225*skyp*PD","7":"202502150216*I*YUL*ABJ*20250223*oojo*AT"},"PriceGrids":{"Grid":[[{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailable":false},{"DirectOutboundAvailabl

In [15]:
import requests
import os

# Define variables for query parameters
departure_location = "SJU"      # fromEntityId: Departure location ID
arrival_location = "EZE"        # toEntityId: Arrival location ID
year_month = "2025-04"          # yearMonth: The month and year for the calendar view

# Base URL for the API endpoint
url = "https://sky-scanner3.p.rapidapi.com/flights/price-calendar-web"

# Set up the query parameters as a dictionary
params = {
    "fromEntityId": departure_location,
    "toEntityId": arrival_location,
    "yearMonth": year_month
}

# Set up the headers including your API key and host
headers = {
    "Content-Type": "application/json",
    "X-RapidAPI-Key": os.getenv("FLIGHTS_RAPIDAPI_KEY"),  # Replace with your actual RapidAPI key
    "X-RapidAPI-Host": os.getenv("FLIGHTS_RAPIDAPI_HOST")
}

# Send the GET request with the parameters and headers
response = requests.get(url, headers=headers, params=params)

# Print the response text
print(response.text)


{"data":{"Outbound":"2025-04","Traces":{"22":"202502150930*I*SJU*EZE*20250413*copa*CM","23":"202502140820*I*SJU*EZE*20250414*copa*CM","24":"202502170125*I*SJU*EZE*20250415*copa*CM","25":"202502140701*I*SJU*EZE*20250416*copa*CM","26":"202502150910*I*SJU*EZE*20250417*copa*CM","27":"202502150911*I*SJU*EZE*20250418*copa*CM","28":"202502150920*I*SJU*EZE*20250430*copa*CM","29":"202502140639*I*SJU*EZE*20250410*copa*CM","10":"202502170101*I*SJU*EZE*20250405*copa*CM","11":"202502150915*I*SJU*EZE*20250427*copa*CM","12":"202502140815*I*SJU*EZE*20250428*copa*CM","13":"202502161529*I*SJU*EZE*20250406*xpus*AV","14":"202502161530*I*SJU*EZE*20250407*copa*CM","15":"202502150915*I*SJU*EZE*20250429*copa*CM","16":"202502141537*I*SJU*EZE*20250420*skyp*NK","17":"202502140818*I*SJU*EZE*20250421*copa*CM","18":"202502160716*I*SJU*EZE*20250408*copa*CM","19":"202502150913*I*SJU*EZE*20250409*uair*UA","0":"202502140711*I*SJU*EZE*20250419*copa*CM","1":"202502150913*I*SJU*EZE*20250422*copa*CM","2":"202502150911*I*SJ

In [ ]:
import json
import pandas as pd
import re

# Parse the JSON string into a Python dictionary
response = json.loads(response.text)

# Extract grid data and trace mappings
grid = response["data"]["PriceGrids"]["Grid"]
traces = response["data"]["Traces"]

# Prepare a list for DataFrame rows
data_rows = []
for row_index, row in enumerate(grid):
    for col_index, cell in enumerate(row):
        direct_available = cell.get("DirectOutboundAvailable")
        # Get indirect flight details (price and trace references)
        indirect = cell.get("Indirect", {})
        price = indirect.get("Price")
        trace_refs = indirect.get("TraceRefs", [])
        
        # Retrieve and format trace details for each reference
        trace_details_list = []
        for ref in trace_refs:
            trace_info = traces.get(ref, "N/A")
            parts = trace_info.split("*")
            if len(parts) >= 7:
                formatted_trace = (
                    f"Dep: {parts[0]}, Type: {parts[1]}, From: {parts[2]}, "
                    f"To: {parts[3]}, Arr: {parts[4]}, Airline: {parts[5]}"
                )
            else:
                formatted_trace = trace_info
            trace_details_list.append(formatted_trace)
        
        data_rows.append({
            "Row": row_index,
            "Column": col_index,
            "Direct Available": direct_available,
            "Indirect Price": price,
            "Trace Refs": ", ".join(trace_refs),
            "Trace Details": " | ".join(trace_details_list)
        })

# Create a Pandas DataFrame from the list of rows
df = pd.DataFrame(data_rows)

# Define a function to parse the trace details
def parse_trace_details(trace_details):
    # Define a regex pattern to capture the desired fields.
    # We ignore the "Type" field since it's not requested.
    pattern = (
        r"Dep:\s*(\d+),\s*Type:\s*\w+,\s*From:\s*(\w+),\s*To:\s*(\w+),\s*Arr:\s*(\d+),\s*Airline:\s*(\w+)"
    )
    match = re.search(pattern, trace_details)
    if match:
        # Extract groups: departure_date, departure_airport, arrival_airport, arrival_date, airline
        dep_date, dep_airport, arr_airport, arr_date, airline = match.groups()
        return pd.Series({
            "departure_date": dep_date,
            "departure_airport": dep_airport,
            "arrival_airport": arr_airport,
            "arrival_date": arr_date,
            "airline": airline
        })
    # If the pattern is not found, return None for each field
    return pd.Series({
        "departure_date": None,
        "departure_airport": None,
        "arrival_airport": None,
        "arrival_date": None,
        "airline": None
    })

# Apply the parsing function to the 'Trace Details' column
parsed_columns = df["Trace Details"].apply(parse_trace_details)

# Concatenate the new columns to the original DataFrame
df = pd.concat([df, parsed_columns], axis=1)

# Display the updated DataFrame
print(df)



    Row  Column  Direct Available  Indirect Price Trace Refs  \
0     0       0             False           290.0          2   
1     0       1             False           290.0          5   
2     0       2             False           321.0          6   
3     0       3             False           290.0          9   
4     0       4             False           356.0         10   
5     0       5             False           269.0         13   
6     0       6             False           290.0         14   
7     0       7             False           290.0         18   
8     0       8             False           409.0         19   
9     0       9             False           492.0         29   
10    0      10             False           405.0         20   
11    0      11             False           376.0         21   
12    0      12             False           442.0         22   
13    0      13             False           293.0         23   
14    0      14             False       

TO DO: 

Refine GET.     